# 3. Architecture

In [ ]:
# ⚔️ Runnable Code
import syft as sy
import hagrid 

sy.requires("==0.7")

## Domain

Launching a domain: Better run the command below in a terminal and with `--dev` flag

In [ ]:
!hagrid launch test_domain domain to docker:8081 --tag=0.7.0 --tail

In [ ]:
hagrid.check("localhost:8083", timeout=120)

In [ ]:
domain_client = sy.login(
    port=8081,
    email="info@openmined.org",
    password="changethis"
)

In [ ]:
domain_client.name

In [ ]:
domain_client.routes[0].connection.base_url

In [ ]:
import numpy as np
x = np.array([1, 2, 3])
x_ptr = x.send(domain_client)
x_ptr

In [ ]:
t = sy.Tensor(x)
t_ptr = t.send(domain_client, tags=["Cool Data"])
t_ptr

## Network

In [ ]:
!hagrid launch test_network network to docker:8082 --tag=0.7.0 --tail

Make a client for the network

In [ ]:
network_client = sy.login(port=8083)

Tell the domain to exchange its `verify_key` with the network

In [ ]:
response = domain_client.networking.initiate_exchange_credentials(
    client=network_client
)

Give the `network` knowledge of the `domain` by what `route` to connect 

In [ ]:
response = domain_client.networking.add_route_for(
    client=network_client,
    source_node_url="http://localhost:8081",
    autodetect=False
)

The purpose of registering a `domain` with a `network` is to provide data search and discovery

In [ ]:
network_client.search(["Cool Data"])

Ask the network to give a list of all its linked domains

In [ ]:
network_client.domains

We can also ask the `network` to proxy our commands via its own `private` connection to this `domain`

In [ ]:
proxy_client = network_client.domains[domain_id]

Check the `domain` store using the new proxy client

In [ ]:
proxy_client.store

Check that the `Cool Data` is available

In [ ]:
cool_data_uid = "fill_me_in"
cool_ptr = proxy_client.store[cool_data_uid]
cool_ptr

Getting the `Cool Data`

In [ ]:
# ⚔️ Runnable Code
try:
    cool_ptr.get(delete_obj=False)
except Exception:
    print("You do not have permission to .get() \
          Object on the node. Please submit a request.")

This should fail because the `proxy_client` does not have permission. However, witht the authorized `domain_client`, we can get the data

In [ ]:
cool_ptr = domain_client.store[cool_data_uid]
cool_data = cool_ptr.get(delete_obj=False)
cool_data